In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/timmmaster')
sys.path.append('../input/tez-lib')

In [ ]:
import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math


In [ ]:
class args:
    batch_size=32
    image_size=384

In [ ]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [ ]:
class PawpularDataset:
    
    def __init__(self,image_paths,dense_features,targets,augmentations):
        self.image_paths=image_paths
        self.dense_features=dense_features
        self.targets=targets
        self.augmentations=augmentations
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,item):
        image=cv2.imread(self.image_paths[item])
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if self.augmentations is not None:
            augmented=self.augmentations(image=image)
            image=augmented["image"]
        image=np.transpose(image,(2,0,1)).astype(np.float32)
        features=self.dense_features[item,:]
        targets=self.targets[item]
        return{
            "image":torch.tensor(image,dtype=torch.float),
            "features":torch.tensor(features,dtype=torch.float),
            "targets":torch.tensor(targets,dtype=torch.float)
        }

In [ ]:
class PawpularModel(tez.Model):
    
    def __init__(self,model_name):
        super().__init__()
        self.model=timm.create_model(model_name,pretrained=False,in_chans=3)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        self.dropout=nn.Dropout(0.2)
        self.dense1=nn.Linear(140,64)
        self.dense2=nn.Linear(64,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense1(x)
        x=self.dense2(x)
        return x,0,{}

In [ ]:
test_aug=albumentations.Compose([
    albumentations.Resize(args.image_size,args.image_size,p=1),
    albumentations.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225],
        max_pixel_value=255.0,
        p=1
    )
],p=1)


In [ ]:
super_final_predictions=[]
for fold_ in range(10):
    model = PawpularModel(model_name="swin_large_patch4_window12_384")
    model.load(f"../input/large-swin-models/model_f{fold_}.bin", device="cuda", weights_only=True)

    df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    test_img_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]

    test_dataset = PawpularDataset(
        image_paths=test_img_paths,
        dense_features=df_test[dense_features].values,
        targets=np.ones(len(test_img_paths)),
        augmentations=test_aug,
    )
    test_predictions = model.predict(test_dataset, batch_size=2*args.batch_size, n_jobs=-1)

    final_test_predictions = []
    for preds in tqdm(test_predictions):
        final_test_predictions.extend(preds.ravel().tolist())

    final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    super_final_predictions.append(final_test_predictions)
    

super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
#super_final_predictions=np.array(super_final_predictions).ravel()
df_test["Pawpularity"]=super_final_predictions
df_test=df_test[["Id","Pawpularity"]]
df_test.to_csv("submission.csv",index=False)